# Graph structures

Following:

http://deeplearning.net/software/theano/tutorial/symbolic_graphs.html

## Theano graphs

The first step in writing Theano code is to write down all the mathematical relations using symbolic placeholders. In this process we use operations like `+`, `-`, `**`, `sum()`, `tanh()`, etc. These are all represented internally as _ops_. An op represents a certain computation on some types of inputs producing some type of output. It is like a _function definition_ in most programming languages.

Theano builds an internal graph structure composed of interconnected _variable_ nodes, _op_ nodes, and _apply_ nodes (which represent the application of an op node to some variables). There is an important difference between the definition of a computation represented by an _op_ and its application to some actual data as represented by the _apply_ node.

See

http://deeplearning.net/software/theano/extending/graphstructures.html#variable

and

http://deeplearning.net/software/theano/extending/graphstructures.html#op

and

http://deeplearning.net/software/theano/extending/graphstructures.html#apply

In [1]:
import theano
import theano.tensor as T
from theano.printing import pydotprint as pdprint

x = T.dmatrix('x')
y = T.dmatrix('y')
z = x + y
pdprint(z, "x_plus_y_eq_z.png", var_with_name_simple=True)

The output file is available at x_plus_y_eq_z.png


<img src='x_plus_y_eq_z.png'>

The graph can be traversed starting from outputs down to its inputs using the owner field.

In [2]:
z.owner

Elemwise{add,no_inplace}(x, y)

In [3]:
z.owner.op.name

'Elemwise{add,no_inplace}'

In [4]:
len(z.owner.inputs)

2

In [5]:
z.owner.inputs[0]

x

In [6]:
z.owner.inputs[1]

y

Consider:

In [7]:
x = theano.tensor.dmatrix('x')
y = x * 2
pdprint(y, "x_times_2_eq_y.png", var_with_name_simple=True)

The output file is available at x_times_2_eq_y.png


<img src='x_times_2_eq_y.png'>

In [8]:
len(y.owner.inputs)

2

In [9]:
y.owner.inputs[0]

x

In [10]:
y.owner.inputs[1]

DimShuffle{x,x}.0

Note the second input isn't `2` - it was first _broadcasted_ by `DimShuffle` to a matrix with the same shape as `x`. 

In [11]:
type(y.owner.inputs[1])

theano.tensor.var.TensorVariable

In [12]:
type(y.owner.inputs[1].owner)

theano.gof.graph.Apply

In [13]:
y.owner.inputs[1].owner.op

In [14]:
y.owner.inputs[1].owner.inputs

[TensorConstant{2}]

This graph structure is illuminating as to how _automatic differentiation_ proceeds, and how symbolic relations may be optimized for performance or stability.

## Automatic differentiation

With a graph structure, computing automatic differentiation is simple. `tensor.grad()` traverses the graph from the outputs back towatds the inputs through all the apply nodes. For each apply node, its op defines how to compute the gradient of the node's outputs with respect to its inputs. (If it doesn't supply this information, the gradient is assumed to be not defined.) Using the chain rule, these gradients can be composed in order to obtain the expression of the gradient of the graph's output with respct to its inputs.

## Optimizations

When compiling a Theano function, what we actually do is give `theano.function` a graph. We optimize that graph by identifying and replacing certain patterns with other specialized patterns that produce the same result, but faster. Optimizations can also identify identical subgraphs and ensure that the same values are not computed twice. We can also reformulate parts of the graph for specific GPUs, etc.

One example is Theano replaces the expression $(xy)/y$ with just $x$.

See also

http://deeplearning.net/software/theano/extending/optimization.html#optimization

and 

http://deeplearning.net/software/theano/optimizations.html#optimizations

### Example

Symbolic programming involves a change of paradigm that will become clearer as we apply it. Consider:

In [15]:
a = theano.tensor.vector("a")     # declare a symbolic variable
b = a + a ** 10                   # build symbolic expression
f = theano.function([a], b)       # compile function
print f([0, 1, 2])

[    0.     2.  1026.]


In [16]:
pdprint(b, outfile='sym_grph_unopt.png', var_with_name_simple=True)
pdprint(f, outfile='sym_grph_opt.png', var_with_name_simple=True)

The output file is available at sym_grph_unopt.png
The output file is available at sym_grph_opt.png


<img src="sym_grph_unopt.png">
<img src="sym_grph_opt.png">